In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from time import sleep
%matplotlib qt 


In [3]:
def Event_Mouse(event, X_Corr, Y_Coor,flag,params) :
	# Left click mouse 
    if event == cv.EVENT_LBUTTONDOWN :
    	# Add seeds
        seeds.append((Y_Coor, X_Corr)) 
        # Draw solid dots 
        cv.circle(Img_MRI, center = (X_Corr, Y_Coor), radius = 2,
                  color = (0, 0, 255), thickness = -1)
        cv.circle(Img_MRI2, center = (X_Corr, Y_Coor), radius = 2,
                  color = (0, 0, 255), thickness = -1)

In [4]:
def Select_seed(img):
    cv.namedWindow('img')
    cv.setMouseCallback('img', Event_Mouse)
    cv.imshow('img', img)

    while True :
        cv.imshow('img', img)
        if cv.waitKey(1) & 0xFF == ord('q') :
            break
    cv.destroyAllWindows()

In [5]:
def joint_histogram(img1 , img2):
    
     fig = plt.figure()
     ax = fig.add_subplot(111, projection='3d')
     x = img1.ravel()
     y = img2.ravel()
     hist, xedges, yedges = np.histogram2d(x, y, bins=20, range=[[0, 255], [0, 255]], density=True)

     xpos, ypos = np.meshgrid(xedges[:-1] + 0.25, yedges[:-1] + 0.25)
     xpos = xpos.flatten('F')
     ypos = ypos.flatten('F')
     zpos = np.zeros_like(xpos)

     dx =10* np.ones_like(zpos)
     dy = dx.copy()
     dz = hist.flatten()

     ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color='b', zsort='average', alpha=0.4)
     ax.set_xlabel('img1')
     ax.set_ylabel('img2')
     plt.show()

In [6]:

Img_MRI2 = cv.imread('Color_MRI2.png',0) #Image to be aligned
Img_MRI = cv.imread('Color_MRI.png',0) #Reference image
height, width = Img_MRI.shape
seeds = []
MRI_copy=Img_MRI.copy()
MRI2_copy=Img_MRI2.copy()

Select_seed(Img_MRI)
Select_seed(Img_MRI2)

Img_MRI_seeds = seeds[0:int(len(seeds)/2)]
Img_MRI2_seeds = seeds[int(len(seeds)/2):len(seeds)]

Img_MRI_seeds = np.float32(Img_MRI_seeds[:])
Img_MRI2_seeds = np.float32(Img_MRI2_seeds[:])

homography, mask = cv.findHomography(Img_MRI_seeds, Img_MRI2_seeds, cv.RANSAC)
transformed_MRI2 = cv.warpPerspective(Img_MRI2, homography, (width, height))

plt.subplot(1,3,1)
plt.imshow(MRI_copy, cmap='gray', vmin = 0, vmax = 255)
plt.axis('off')
plt.title('Reference image')
plt.subplot(1,3,2)
plt.imshow(MRI2_copy, cmap='gray', vmin = 0, vmax = 255)
plt.axis('off')
plt.title('Image to be aligned')
plt.subplot(1,3,3)
plt.imshow(transformed_MRI2, cmap='gray', vmin = 0, vmax = 255)
plt.axis('off')
plt.title('aligned image')
plt.tight_layout()
plt.show()

In [ ]:
joint_histogram(Img_MRI, Img_MRI2)
joint_histogram(Img_MRI, transformed_MRI2)
joint_histogram(Img_MRI, Img_MRI)
